In [1]:
import torch
import os
torch.__version__

'1.11.0'

In [3]:
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
albu_train_transforms = [
    #     dict(
    #         type='ShiftScaleRotate',
    #         shift_limit=0.0625,
    #         scale_limit=0.0,
    #         rotate_limit=0.1,
    #         interpolation=1,
    #         p=0.2),
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.2),
    dict(
        type='OneOf',
        transforms=[
            dict(
                type='RGBShift',
                r_shift_limit=10,
                g_shift_limit=10,
                b_shift_limit=10,
                p=1.0),
            dict(
                type='HueSaturationValue',
                hue_shift_limit=20,
                sat_shift_limit=30,
                val_shift_limit=20,
                p=1.0)
        ],
        p=0.1),
    #     dict(type='JpegCompression', quality_lower=85, quality_upper=95, p=0.2),
    #     dict(type='ChannelShuffle', p=0.1),
    dict(
        type='OneOf',
        transforms=[
            dict(type='Blur', blur_limit=3, p=1.0),
            dict(type='MedianBlur', blur_limit=3, p=1.0)
        ],
        p=0.3),
]
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='Pad', size_divisor=32),
    #     dict(type='Mosaic', img_scale=(1000, 800), pad_val=114.0),
    #     dict(
    #         type='RandomAffine',
    #         scaling_ratio_range=(0.85, 1.15),
    #         border=(-1000 // 2, -800 // 2)),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='DefaultFormatBundle'),
    dict(
        type='Collect',
        keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'],
        meta_keys=('filename', 'ori_shape', 'img_shape', 'img_norm_cfg',
                   'pad_shape', 'scale_factor'))
]

test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip', flip_ratio=0.5),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

In [4]:
import mmcv
from mmcv.runner import load_checkpoint
from mmdet.apis import inference_detector, show_result_pyplot
from mmdet.models import build_detector
from mmdet.apis import set_random_seed


def set_config():
    # 사용할 모델 초기 py 설정
    config = '/home/eunsoo/dl/mmdetection/configs/competition/lg2.py'

    # config 불러오기
    cfg = mmcv.Config.fromfile(config)
    cfg.model.backbone = dict(
        type='ResNeXt',
        depth=101,
        groups=64,
        base_width=4,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        #         frozen_stages=-1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(
              type='Pretrained', checkpoint='open-mmlab://resnext101_64x4d'))

    cfg.dataset_type = 'CocoDataset'

    data_root = '/home/eunsoo/dl/mmdetection/data/dataset/'

    # class 설정
    cfg.classes = ('Normal',)

    # train, val, test 데이터셋에 대한 type, data_root, ann_file, img_prefix 설정

    cfg.data.samples_per_gpu = 1
    cfg.data.workers_per_gpu = 0

    cfg.data.train.data_root = data_root
    cfg.data.train.ann_file = 'label(polygon)_train.json'
    cfg.data.train.img_prefix = 'train/'
    cfg.data.train.classes = cfg.classes

    cfg.data.val.data_root = data_root
    cfg.data.val.ann_file = 'label(polygon)_train.json'
    cfg.data.val.img_prefix = 'train/'
    cfg.data.val.classes = cfg.classes

    cfg.data.test.data_root = data_root
    cfg.data.test.ann_file = 'test.json'
    cfg.data.test.img_prefix = 'test/'
    cfg.data.test.classes = cfg.classes

    cfg.train_pipeline = train_pipeline
    cfg.test_pipeline = test_pipeline
    cfg.data.train.pipeline = train_pipeline
    cfg.data.val.pipeline = test_pipeline
    cfg.data.test.pipeline = test_pipeline

    cfg.model.roi_head.bbox_head[0].num_classes = 1
    cfg.model.roi_head.bbox_head[1].num_classes = 1
    cfg.model.roi_head.bbox_head[2].num_classes = 1

    cfg.model.roi_head.mask_head.num_classes = 1
    cfg.model.roi_head.glbctx_head.num_classes = 1

    cfg.load_from = '/home/eunsoo/dl/competition/lg/scnet_rev3/scnet_rev3/latest.pth'

    cfg.checkpoint_config = dict(
        interval=3, out_dir='/home/eunsoo/dl/competition/lg/scanet_last_rev4/')

    cfg.work_dir = '/home/eunsoo/dl/competition/lg/scanet_last_rev4'
#     cfg.optimizer = dict(type='AdamW', lr=0.0025/8,
#                          weight_decay=0.0001/8)
    cfg.evaluation.metric = ['bbox', 'segm']
    cfg.optimizer = dict(type='SGD', lr=0.0025,
                         momentum=0.9, weight_decay=0.0001)
    cfg.evaluation.save_best = 'auto'
    cfg.evaluation.interval = 6

    cfg.runner = dict(type='EpochBasedRunner', max_epochs=60)

    cfg.auto_scale_lr = dict(enable=False, base_batch_size=4)

    cfg.seed = 0
    cfg.gpu_ids = range(1)
    cfg.device = 'cuda'
#TODO , hook
# https://mmdetection.readthedocs.io/en/dev/tutorials/how_to.html
    return cfg

In [5]:
cfg = set_config()
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='Pad', size_divisor=32),
    dict(
        type='Albu',
        transforms=[
            dict(
                type='RandomBrightnessContrast',
                brightness_limit=[0.1, 0.3],
                contrast_limit=[0.1, 0.3],
                p=0.2),
            dict(
                type='OneOf',
                transforms=[
                    dict(
                        type='RGBShift',
                        r_shift_limit=10,
                        g_shift_limit=10,
                        b_shift_limit=10,
                        p=1.0),
                    dict(
                        type='HueSaturationValue',
    

In [6]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

# dataset 생성 및 model 설정

datasets = [build_dataset(cfg.data.train)]
model = build_detector(cfg.model)
model.CLASSES = datasets[0].CLASSES

# 훈련
train_detector(model, datasets, cfg, distributed=False, validate=True)

loading annotations into memory...
Done (t=0.56s)
creating index...
index created!


2022-08-08 10:28:30,452 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...


2022-08-08 10:28:30,982 - mmdet - INFO - load checkpoint from local path: /home/eunsoo/dl/competition/lg/scnet_rev3/scnet_rev3/latest.pth


Done (t=0.50s)
creating index...
index created!


...

Loading and preparing results...
DONE (t=0.30s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=38.19s).
Accumulating evaluation results...


/home/eunsoo/anaconda3/envs/pytorch/lib/python3.7/site-packages/pycocotools-2.0.4-py3.7-linux-x86_64.egg/pycocotools/cocoeval.py:378: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  tp_sum = np.cumsum(tps, axis=1).astype(dtype=np.float)
2022-08-08 13:31:34,466 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.676
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.940
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.829
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.476
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000

DONE (t=0.59s).


2022-08-08 13:31:36,432 - mmdet - INFO - Now best checkpoint is saved as best_bbox_mAP_epoch_18.pth.
2022-08-08 13:31:36,432 - mmdet - INFO - Best bbox_mAP is 0.7730 at 18 epoch.
2022-08-08 13:31:36,433 - mmdet - INFO - Epoch(val) [18][520]	bbox_mAP: 0.7730, bbox_mAP_50: 0.9400, bbox_mAP_75: 0.8880, bbox_mAP_s: 0.5720, bbox_mAP_m: 0.8100, bbox_mAP_l: 0.9290, bbox_mAP_copypaste: 0.773 0.940 0.888 0.572 0.810 0.929, segm_mAP: 0.6760, segm_mAP_50: 0.9400, segm_mAP_75: 0.8290, segm_mAP_s: 0.4760, segm_mAP_m: 0.7060, segm_mAP_l: 0.8400, segm_mAP_copypaste: 0.676 0.940 0.829 0.476 0.706 0.840
2022-08-08 13:32:30,733 - mmdet - INFO - Epoch [19][50/520]	lr: 2.500e-05, eta: 6:15:06, time: 1.085, data_time: 0.313, memory: 6845, loss_rpn_cls: 0.0088, loss_rpn_bbox: 0.0243, loss_glbctx: 0.0000, s0.loss_cls: 0.0610, s0.acc: 97.4844, s0.loss_bbox: 0.0444, s1.loss_cls: 0.0161, s1.acc: 98.8162, s1.loss_bbox: 0.0534, s2.loss_cls: 0.0094, s2.acc: 98.5913, s2.loss_bbox: 0.0414, loss_mask: 0.2893, loss: 0

KeyboardInterrupt: 

In [7]:
cfg = set_config()
cfg.gpu_ids = 0

In [8]:
import argparse
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.cnn import fuse_conv_bn
from mmcv.runner import get_dist_info, init_dist, load_checkpoint, wrap_fp16_model

from mmdet.apis import multi_gpu_test, single_gpu_test
from mmdet.datasets import build_dataloader, build_dataset,  replace_ImageToTensor
from mmdet.models import build_detector
from mmdet.utils import build_ddp, build_dp, compat_cfg, get_device, replace_cfg_vals, setup_multi_processes, update_data_root


# 필요한 파라미터 설정
checkpoint = '/home/eunsoo/dl/competition/lg/scanet_last_rev4/scanet_last_rev4/latest.pth'  # 훈련시킨 모델 가중치
show = False
show_dir = None
show_score_thr = 0.5
# json 출력 저장위치
eval_options = {
    'jsonfile_prefix': '/home/eunsoo/dl/competition/lg/scanet_last_rev4/output/predic'}
format_only = True
out = None


## 여기서 부터 기존의 mmdetection/tools/test.py 를 변경한 코드 ##

cfg = replace_cfg_vals(cfg)
update_data_root(cfg)
cfg = compat_cfg(cfg)
setup_multi_processes(cfg)


# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

if 'pretrained' in cfg.model:
    cfg.model.pretrained = None
elif 'init_cfg' in cfg.model.backbone:
    cfg.model.backbone.init_cfg = None

if cfg.model.get('neck'):
    if isinstance(cfg.model.neck, list):
        for neck_cfg in cfg.model.neck:
            if neck_cfg.get('rfp_backbone'):
                if neck_cfg.rfp_backbone.get('pretrained'):
                    neck_cfg.rfp_backbone.pretrained = None
    elif cfg.model.neck.get('rfp_backbone'):
        if cfg.model.neck.rfp_backbone.get('pretrained'):
            cfg.model.neck.rfp_backbone.pretrained = None


cfg.gpu_ids = [cfg.gpu_ids]
cfg.device = get_device()


distributed = False


test_dataloader_default_args = dict(
    samples_per_gpu=1, workers_per_gpu=2, dist=distributed, shuffle=False)

# in case the test dataset is concatenated
if isinstance(cfg.data.test, dict):
    cfg.data.test.test_mode = True
    if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
        # Replace 'ImageToTensor' to 'DefaultFormatBundle'
        cfg.data.test.pipeline = replace_ImageToTensor(
            cfg.data.test.pipeline)
elif isinstance(cfg.data.test, list):
    for ds_cfg in cfg.data.test:
        ds_cfg.test_mode = True
    if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
        for ds_cfg in cfg.data.test:
            ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

test_loader_cfg = {
    **test_dataloader_default_args,
    **cfg.data.get('test_dataloader', {})
}

rank, _ = get_dist_info()


# build the dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, **test_loader_cfg)

# build the model and load checkpoint
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))

fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')


if 'CLASSES' in checkpoint.get('meta', {}):
    model.CLASSES = checkpoint['meta']['CLASSES']
else:
    model.CLASSES = dataset.CLASSES

if not distributed:
    model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)
    outputs = single_gpu_test(model, data_loader, show, show_dir,
                              show_score_thr)


rank, _ = get_dist_info()
if rank == 0:
    if out:
        print(f'\nwriting results to {out}')
        mmcv.dump(outputs, out)
    kwargs = {} if eval_options is None else eval_options
    if format_only:
        dataset.format_results(outputs, **kwargs)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: /home/eunsoo/dl/competition/lg/scanet_last_rev4/scanet_last_rev4/latest.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 350/350, 2.2 task/s, elapsed: 162s, ETA:     0s

In [27]:
import argparse
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.cnn import fuse_conv_bn
from mmcv.runner import get_dist_info, init_dist, load_checkpoint, wrap_fp16_model

from mmdet.apis import multi_gpu_test, single_gpu_test
from mmdet.datasets import build_dataloader, build_dataset,  replace_ImageToTensor
from mmdet.models import build_detector
from mmdet.utils import build_ddp, build_dp, compat_cfg, get_device, replace_cfg_vals, setup_multi_processes, update_data_root


# 필요한 파라미터 설정
checkpoint = '/home/eunsoo/dl/competition/lg/scanet_last_rev/scanet_last_rev/epoch_100.pth'  # 훈련시킨 모델 가중치
show = False
show_dir = None
show_score_thr = 0.2
# json 출력 저장위치
eval_options = {
    'jsonfile_prefix': '/home/eunsoo/dl/competition/lg/scanet_last_rev/output/predic_rev'}
format_only = True
out = None


## 여기서 부터 기존의 mmdetection/tools/test.py 를 변경한 코드 ##

cfg = replace_cfg_vals(cfg)
update_data_root(cfg)
cfg = compat_cfg(cfg)
setup_multi_processes(cfg)


# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

if 'pretrained' in cfg.model:
    cfg.model.pretrained = None
elif 'init_cfg' in cfg.model.backbone:
    cfg.model.backbone.init_cfg = None

if cfg.model.get('neck'):
    if isinstance(cfg.model.neck, list):
        for neck_cfg in cfg.model.neck:
            if neck_cfg.get('rfp_backbone'):
                if neck_cfg.rfp_backbone.get('pretrained'):
                    neck_cfg.rfp_backbone.pretrained = None
    elif cfg.model.neck.get('rfp_backbone'):
        if cfg.model.neck.rfp_backbone.get('pretrained'):
            cfg.model.neck.rfp_backbone.pretrained = None


cfg.gpu_ids = [cfg.gpu_ids]
cfg.device = get_device()


distributed = False


test_dataloader_default_args = dict(
    samples_per_gpu=1, workers_per_gpu=2, dist=distributed, shuffle=False)

# in case the test dataset is concatenated
if isinstance(cfg.data.test, dict):
    cfg.data.test.test_mode = True
    if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
        # Replace 'ImageToTensor' to 'DefaultFormatBundle'
        cfg.data.test.pipeline = replace_ImageToTensor(
            cfg.data.test.pipeline)
elif isinstance(cfg.data.test, list):
    for ds_cfg in cfg.data.test:
        ds_cfg.test_mode = True
    if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
        for ds_cfg in cfg.data.test:
            ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

test_loader_cfg = {
    **test_dataloader_default_args,
    **cfg.data.get('test_dataloader', {})
}

rank, _ = get_dist_info()


# build the dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, **test_loader_cfg)

# build the model and load checkpoint
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))

fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')


if 'CLASSES' in checkpoint.get('meta', {}):
    model.CLASSES = checkpoint['meta']['CLASSES']
else:
    model.CLASSES = dataset.CLASSES

if not distributed:
    model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)
    outputs = single_gpu_test(model, data_loader, show, show_dir,
                              show_score_thr)


rank, _ = get_dist_info()
if rank == 0:
    if out:
        print(f'\nwriting results to {out}')
        mmcv.dump(outputs, out)
    kwargs = {} if eval_options is None else eval_options
    if format_only:
        dataset.format_results(outputs, **kwargs)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: /home/eunsoo/dl/competition/lg/scanet_last_rev/scanet_last_rev/epoch_100.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 350/350, 0.2 task/s, elapsed: 1571s, ETA:     0s

In [35]:
# from mmdet.apis import init_detector, inference_detector
# from mmdet.apis import show_result_pyplot

# model = init_detector(
#     config=cfg, checkpoint='/home/eunsoo/dl/competition/lg/scnet_rev2/scnet_rev2/latest.pth')
# img = '/home/eunsoo/dl/mmdetection/data/dataset/test/0080.jpg'
# results = inference_detector(model, img)

# show_result_pyplot(model, img, results)
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1000, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip', flip_ratio=0.5),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]
cfg.test_pipeline = test_pipeline
cfg.data.test.pipeline = test_pipeline

In [38]:
import argparse
import os
import os.path as osp
import time
import warnings

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.cnn import fuse_conv_bn
from mmcv.runner import get_dist_info, init_dist, load_checkpoint, wrap_fp16_model

from mmdet.apis import multi_gpu_test, single_gpu_test
from mmdet.datasets import build_dataloader, build_dataset,  replace_ImageToTensor
from mmdet.models import build_detector
from mmdet.utils import build_ddp, build_dp, compat_cfg, get_device, replace_cfg_vals, setup_multi_processes, update_data_root


# 필요한 파라미터 설정
checkpoint = '/home/eunsoo/dl/competition/lg/scanet_last_rev3/best_bbox_mAP_epoch_36.pth'   # 훈련시킨 모델 가중치
show = False
show_dir = None
show_score_thr = 0.3
# json 출력 저장위치
eval_options = {
    'jsonfile_prefix': '/home/eunsoo/dl/competition/lg/scanet_last_rev3/output/predic_rev'}
format_only = True
out = None


## 여기서 부터 기존의 mmdetection/tools/test.py 를 변경한 코드 ##

cfg = replace_cfg_vals(cfg)
update_data_root(cfg)
cfg = compat_cfg(cfg)
setup_multi_processes(cfg)


# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

if 'pretrained' in cfg.model:
    cfg.model.pretrained = None
elif 'init_cfg' in cfg.model.backbone:
    cfg.model.backbone.init_cfg = None

if cfg.model.get('neck'):
    if isinstance(cfg.model.neck, list):
        for neck_cfg in cfg.model.neck:
            if neck_cfg.get('rfp_backbone'):
                if neck_cfg.rfp_backbone.get('pretrained'):
                    neck_cfg.rfp_backbone.pretrained = None
    elif cfg.model.neck.get('rfp_backbone'):
        if cfg.model.neck.rfp_backbone.get('pretrained'):
            cfg.model.neck.rfp_backbone.pretrained = None


cfg.gpu_ids = [cfg.gpu_ids]
cfg.device = get_device()


distributed = False


test_dataloader_default_args = dict(
    samples_per_gpu=1, workers_per_gpu=2, dist=distributed, shuffle=False)

# in case the test dataset is concatenated
if isinstance(cfg.data.test, dict):
    cfg.data.test.test_mode = True
    if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
        # Replace 'ImageToTensor' to 'DefaultFormatBundle'
        cfg.data.test.pipeline = replace_ImageToTensor(
            cfg.data.test.pipeline)
elif isinstance(cfg.data.test, list):
    for ds_cfg in cfg.data.test:
        ds_cfg.test_mode = True
    if cfg.data.test_dataloader.get('samples_per_gpu', 1) > 1:
        for ds_cfg in cfg.data.test:
            ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)

test_loader_cfg = {

    **test_dataloader_default_args,
    **cfg.data.get('test_dataloader', {})
}

rank, _ = get_dist_info()


# build the dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, **test_loader_cfg)

# build the model and load checkpoint
cfg.model.train_cfg = None
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))

fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)
checkpoint = load_checkpoint(model, checkpoint, map_location='cpu')


if 'CLASSES' in checkpoint.get('meta', {}):
    model.CLASSES = checkpoint['meta']['CLASSES']
else:
    model.CLASSES = dataset.CLASSES

if not distributed:
    model = build_dp(model, cfg.device, device_ids=cfg.gpu_ids)
    outputs = single_gpu_test(model, data_loader, show, show_dir,
                              show_score_thr)


rank, _ = get_dist_info()
if rank == 0:
    if out:
        print(f'\nwriting results to {out}')
        mmcv.dump(outputs, out)
    kwargs = {} if eval_options is None else eval_options
    if format_only:
        dataset.format_results(outputs, **kwargs)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
load checkpoint from local path: /home/eunsoo/dl/competition/lg/scanet_last_rev3/best_bbox_mAP_epoch_36.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 350/350, 2.1 task/s, elapsed: 168s, ETA:     0s